# GPU memory usage for deep learning image processing

### 目的
DLによる画像処理について、モデルの学習と実行に必要なGPUメモリー容量の理論値と実測値の差を検証する。

### 方法
以下の４つの項目について、メモリの使用量の理論値と実測値を比較する。
   
1. 変数
2. Convolutional Neural Network
3. VGG16
4. ResNet50
5. Vision Transformer
6. UNETR

メモリの使用量の実測値は、pytorchもしくはtensorflowの関数と、nvml（nvidia  management library）の関数で測定する。
pytorchやtensorflowの関数ではプログラム中で確保された容量が測定される。一方、nvmlではプログラム実行に必要なpythonやcudaなどのオーバーヘッドを含んだ容量が測定される。
変数とUNETRについては２つの実装方法（pytorch, tensorflow）で実測値を測定する。どちらの実装方法でも実測値に大きな差はないことが期待される。

### 結果
|  | 理論値[MiB] | 実測値[MiB] | 誤差[%] |
|---|---|---|---|
| 変数 | 1024 | 1024 | 0 |
| CNN | 636 | 531 | 19.8 |
| VGG16 | 15929 | 17232 | 7.6 |
| ResNet50 | 11378 | 11738 | 3.1 |
| ViT | 13290 | 17217 | 22.8 |
| UNETR |  |  |  |

#### 検証環境
- python 3.10
- tensorflow 2.13.0
- torch 2.0.1
- NVIDIA driver 530.30.02
- CUDA toolkit 11.8
- cuDNN 8.9

#### 参考文献
1. [Estimating GPU Memory Consumption of Deep Learning Models](https://2020.esec-fse.org/details/esecfse-2020-industry-papers/5/Estimating-GPU-Memory-Consumption-of-Deep-Learning-Models), [video](https://dl.acm.org/doi/10.1145/3368089.3417050)
2. [A comprehensive guide to memory usage in PyTorch](https://medium.com/deep-learning-for-protein-design/a-comprehensive-guide-to-memory-usage-in-pytorch-b9b7c78031d3)

## 変数
float32型（4byte）で0.25GiB次元の変数を考える。この変数は理論値で1GiBのサイズである。

変数を定義または削除したときのメモリの使用量を確認して、実測値を求める。
まず、Pytorchでの実測値を求める。

In [1]:
import pynvml
import torch


def print_memory_torch(prefix: str):
    """Print memory usage.
    """    
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)    
    memory_al = torch.cuda.memory_allocated()
    memory_res = torch.cuda.memory_reserved()
    memory_maxal = torch.cuda.max_memory_allocated()

    print(f"{prefix}: allocated = {memory_al/1024**2:.1f} MiB, "
        f"reserved = {memory_res/1024**2:.1f}MiB, "
        f"max allocated = {memory_maxal/1024**2:.1f} MiB, "
        f"used = {info.used/1024**2:.1f} MiB")


# Define a variable with 1GiB.
dim = 1024**3//4
var_cpu = torch.zeros(dim, dtype=torch.float32)
print(f"var_cpu dtype: {var_cpu.dtype}, dim: {dim/1024**2}MiB")

torch.cuda.reset_peak_memory_stats()
print_memory_torch("Initial")

# Copy the variable to gpu.
var_gpu = var_cpu.to("cuda")
print_memory_torch("Define")

# Delete the variable from gpu.
del var_gpu
print_memory_torch("Delete")

# Release cached memory.
torch.cuda.empty_cache()
print_memory_torch("Release")


var_cpu dtype: torch.float32, dim: 256.0MiB
Initial: allocated = 0.0 MiB, reserved = 0.0MiB, max allocated = 0.0 MiB, used = 673.6 MiB
Define: allocated = 1024.0 MiB, reserved = 1024.0MiB, max allocated = 1024.0 MiB, used = 2497.6 MiB
Delete: allocated = 0.0 MiB, reserved = 1024.0MiB, max allocated = 1024.0 MiB, used = 2497.6 MiB
Release: allocated = 0.0 MiB, reserved = 0.0MiB, max allocated = 1024.0 MiB, used = 1473.6 MiB


変数を定義した後に1024MiB=1GiBのVRAMが確保されたので、実測値は理論値と一致した。
delで変数を削除したあともメモリはreservedとして確保されており、torch.cuda.empty_cache()によって完全にメモリが解放された。

nvmlのusedの結果から、実際には変数のサイズ以上のメモリが確保され、変数を削除した後にもオーバーヘッド分が残っていた。

次にTensorFlowでの実測値を求める。
Tensorflowはデフォルトでプログラム開始時にメモリを最大限確保する。必要なメモリだけを確保するために、memory growthを有効にする。

In [2]:
import tensorflow as tf
import numpy as np


def print_memory_tf(prefix: str):
    """Print memory usage.
    """
    pynvml.nvmlInit()
    handle = pynvml.nvmlDeviceGetHandleByIndex(0)
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)  
    memory_info = tf.config.experimental.get_memory_info("GPU:0")

    print(f"{prefix}: current = {memory_info['current']/1024**2:.1f} MiB, "
        f"peak = {memory_info['peak']/1024**2:.1f}MiB, "
        f"used = {info.used/1024**2:.1f} MiB")


# Enabled memory growth.
devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(devices[0], True)

# Define a variable with 1GiB.
dim = 1024**3//4
var_cpu = np.zeros(dim, dtype=np.float32)
print(f"var_cpu dtype: {var_cpu.dtype}, dim: {dim/1024**2}MiB")

tf.config.experimental.reset_memory_stats("GPU:0")
print_memory_tf("Initail")

# Copy the variable to gpu.
with tf.device("GPU"):
    var_gpu = tf.constant(var_cpu)
print_memory_tf("Define")

# Delete the variable from gpu.
del var_gpu
print_memory_tf("Delete")

2023-08-09 20:23:10.003822: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-09 20:23:10.024979: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 20:23:10.430311: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-09 20:23:10.910824: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.

var_cpu dtype: float32, dim: 256.0MiB
Initail: current = 0.0 MiB, peak = 0.0MiB, used = 1650.2 MiB


2023-08-09 20:23:10.945510: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-09 20:23:10.945638: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-09 20:23:10.945710: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Define: current = 1024.0 MiB, peak = 1024.0MiB, used = 3717.6 MiB
Delete: current = 0.0 MiB, peak = 1024.0MiB, used = 3717.6 MiB


変数を定義した後に1GiBのVRAMが確保されたので、実測値は理論値と一致した。
オーバーヘッドも含めると、実際には変数のサイズ以上のメモリが確保され、変数を削除したあとも解放されなかった。

## Convolutional Neural Network
画像分類をする単純なCNNについて、メモリー使用量の理論値と実測値を比較する。

データセットは疑似乱数で作成する。
入力画像は3x224x224次元として、1280枚を用意する。
分類クラスは10個として、one-hot encodingで表現する。
バッチサイズは128とする。

CNNは、畳み込み層、平均プーリング層、全結合層を1つずつ持つ構造とする。
畳み込み層は、カーネル3x3、チャンネル数8、ストライド1、パディングなし、バイアスありとする。
平均プーリング層は、カーネル2x2、ストライド2、パディングなしとする。
全結合層は10次元の出力である。
学習の最適化にはSGDを用いる。
数値の精度はPytorchのデフォルトであるfloat32とする。

このCNNに必要なメモリの理論値を求める。
参考文献1,2によれば、学習と推論に必要なメモリは各々以下のようになる。

$$
\begin{align*}
\text{usage for training}   &= \text{data} + \text{weight} + \text{forward output} + \text{weight gradient} + \text{output gradient} \\
                            &= D + W + O * b + W * (d + m) + (O + D) \\
\text{usage for inference}  &= \text{data} + \text{weight} + \text{forward output} \\
                            &= D + W + O \\
\end{align*}
$$

ここで、  
$D$: ミニバッチのデータセットのメモリ使用量  
$W$: 学習パラメーターのメモリ使用量  
$O$: 中間層の出力のメモリ使用量  
$b$: 推論時に中間層の出力のみ半精度にするMixed Precisionを利用する場合は0.5、そうでない場合は1  
$d$: 複数のGPUで実行するDistributed Data Parallelを利用する場合は2、そうでない場合は1  
$m$: 最適化で使うモーメントの数（SGD: 0, Adagrad, RMSprop: 1, Adam: 2）

実際にはこれに加えて、GPUでの計算に必要なメモリー（CUDA context、cuDNN Workspace）とメモリ管理の最適化のための余剰メモリー（Internal Tensor Fragmentation）が使用される。文献1によれば、この追加分はライブラリーのバージョンやモデルによって変わるが、0.5GB程度である。

今回のCNNの場合は、メモリー使用量の理論値は以下のように求まる。

In [3]:
from torch import optim

class Config:
    def __init__(self):
        self.dim_input = [3,224,224]
        self.dim_output = 10
        self.datasize = 1280
        self.batchsize = 128
        self.num_epochs = 3
        self.lr = 1e-2
        self.device = 'cuda'
        self.optim = optim.SGD
        self.moment = 0         # SGD: 0, Adagrad, RMSprop: 1, Adam: 2
        self.ddp = 1            # Distributed data parallel: 2, Not: 1
        self.mixed_pre = 1      # Mixed precision: 0.5, Not: 1


conf = Config()

In [4]:
def print_memory_estimate1(
    dim_input: list, 
    dim_output: list, 
    num_param: int, 
    num_hidden_output: int, 
    moment: int, 
    ddp: int = 1, 
    mixed_pre: float = 1):
    '''Print theoretical memory usage.
    
    Parameters
    ----------
    dim_input: Shape of input data including batch size. e.g. [batch size, channel, width, height]
    dim_output: Shape of output data.
    num_param: Number of trainable parameters.
    num_hidden_output: Total number of hidden layer's output. Don't include in-place hidden layer that require no additional memory (Activations, Normalizations).
    moment: Moment use for optimization. SGD: 0, Adagrad, RMSprop: 1, Adam: 2
    ddp: Multiple GPU use. Distributed data parallel: 2, Not: 1
    mixed_pre: Forward outputs memory saving by Mixed precision: 0.5, Not: 1
    '''
    mem_data = (np.prod(dim_input) + np.prod(dim_output)) * 4
    mem_weight = num_param * 4
    mem_weight_grad = mem_weight * (ddp + moment)
    mem_forward_output = num_hidden_output * 4 * mixed_pre
    mem_output_gradient = mem_forward_output + mem_data
    mem_training = mem_data + mem_weight + mem_forward_output + mem_weight_grad + mem_output_gradient
    mem_inference = mem_data + mem_weight + mem_forward_output

    print(f"Data(MiB): {mem_data/1024**2:.1f}")
    print(f"Weight(MiB): {mem_weight/1024**2:.1f}")
    print(f"Forward output(MiB): {mem_forward_output/1024**2:.1f}")
    print(f"Weight gradient(MiB): {mem_weight_grad/1024**2:.1f}")
    print(f"Output gradient(MiB): {mem_output_gradient/1024**2:.1f}")
    print(f"Total for training(MiB): {mem_training/1024**2:.1f}")
    print(f"Total for inference(MiB): {mem_inference/1024**2:.1f}")


num_param = 3*3*3*8+8 + 8*((conf.dim_input[1]-2)//2)*((conf.dim_input[2]-2)//2)*10 + 10
num_output_shape = np.prod([conf.batchsize, 8, (conf.dim_input[1]-2), (conf.dim_input[2]-2)]) \
                 + np.prod([conf.batchsize, 8, ((conf.dim_input[1]-2)//2), ((conf.dim_input[2]-2)//2)])

print_memory_estimate1([conf.batchsize] + conf.dim_input, conf.dim_output,
                        num_param, num_output_shape, conf.moment, conf.ddp, conf.mixed_pre)

Data(MiB): 73.5
Weight(MiB): 3.8
Forward output(MiB): 240.6
Weight gradient(MiB): 3.8
Output gradient(MiB): 314.1
Total for training(MiB): 635.8
Total for inference(MiB): 317.9


よって、理論値は636MiBである。

次にpytorchでCNNを実行したときのメモリ使用量の実測値を調べる。

In [5]:
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm


class CNN(nn.Module):    
    def __init__(self, dim_c: int, dim_h: int, dim_w: int):
        super().__init__()
        self.conv = nn.Conv2d(in_channels=dim_c, out_channels=8, kernel_size=3)
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(8*((dim_h-2)//2)*((dim_w-2)//2), 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

def train(
    model:nn.Module, 
    dim_input: list, 
    batchsize: int, 
    epoch: int, 
    optimizer: optim.Optimizer = optim.SGD, 
    device: str = "cuda"):
    """Train model using random dataset.
    
    Parameters
    ----------
    model: 
    dim_input: Shape of input data including data size. e.g. [data size, channel, width, height]
    batchsize:
    epoch:
    optimizer:
    device: 
    """
    
    torch.cuda.empty_cache()
    torch.cuda.reset_peak_memory_stats()
    print_memory_torch("Initial")

    model.to(device)
    print_memory_torch("Model")

    data = [[torch.randn([batchsize, dim_input[1], dim_input[2], dim_input[3]]), 
             torch.randn(batchsize, 10)] 
             for _ in range(dim_input[0]//batchsize)]

    criterion = F.cross_entropy
    opt = optimizer(model.parameters(), lr=0.01)
    for ep in range(epoch):
        model.train()
        with tqdm(data) as pbar:
            pbar.set_description(f'[Epoch {ep + 1}]')
            for x, y in pbar:
                x = x.to(device)
                y = y.to(device)
                
                opt.zero_grad()
                y_pred = model(x)
                loss = criterion(y_pred, y)
                loss.backward()
                opt.step()
            
        print_memory_torch("Train")
    print_memory_torch("Final")


model_cnn = CNN(conf.dim_input[0], conf.dim_input[1], conf.dim_input[2])
train(model_cnn, [conf.datasize] + conf.dim_input, conf.batchsize, conf.num_epochs,
      conf.optim, conf.device)

Initial: allocated = 0.0 MiB, reserved = 0.0MiB, max allocated = 0.0 MiB, used = 3717.6 MiB
Model: allocated = 3.8 MiB, reserved = 22.0MiB, max allocated = 3.8 MiB, used = 3739.6 MiB


[Epoch 1]: 100%|██████████| 10/10 [00:00<00:00, 10.97it/s]


Train: allocated = 97.8 MiB, reserved = 604.0MiB, max allocated = 530.9 MiB, used = 5001.4 MiB


[Epoch 2]: 100%|██████████| 10/10 [00:00<00:00, 108.79it/s]


Train: allocated = 97.8 MiB, reserved = 604.0MiB, max allocated = 530.9 MiB, used = 5001.4 MiB


[Epoch 3]: 100%|██████████| 10/10 [00:00<00:00, 108.70it/s]


Train: allocated = 97.8 MiB, reserved = 604.0MiB, max allocated = 530.9 MiB, used = 5001.4 MiB
Final: allocated = 97.8 MiB, reserved = 604.0MiB, max allocated = 530.9 MiB, used = 5001.4 MiB


モデルを用意したときの実測値は3.8MiBであり、理論値と一致した。
学習終了時の実測ピーク値は531MiBであり、理論値と19.8%の誤差があった。



pytorchでは、torchinfoを使うとネットワーク構造やパラメータ数、メモリー使用量の情報を取得・表示できる。この機能は理論値を計算する際に便利であるが、メモリー使用量の計算は上記理論式と異なる。

具体的には、メモリー使用量＝入力データ＋順伝搬/逆伝搬サイズ＋Weightとしている。順伝搬/逆伝搬サイズForward/backward pass sizeは、学習する層の出力サイズの合計を求め、gradientを考慮してその２倍をメモリー使用量としている。よって、上記理論式とは全く異なる。
また、メモリー使用量の表示がMiBではなくMBである。（1MB = 1000^2byte, 1MiB = 1,024^2byte）

今回のCNNでのtorchinfoの表示は以下のようになる。

In [6]:
from torchinfo import summary

result = summary(model_cnn, [conf.batchsize] + conf.dim_input,
        col_names=("output_size", "num_params", "kernel_size"))
print(result)

# Confirm result
num_input = conf.batchsize * conf.dim_input[0] * conf.dim_input[1] * conf.dim_input[2]
num_param = conf.dim_input[0]*3*3*8 + 8 + 8*((conf.dim_input[1]-2)//2)*((conf.dim_input[2]-2)//2)*conf.dim_output + conf.dim_output
num_hidden_output = np.prod([conf.batchsize, 8, conf.dim_input[1]-2, conf.dim_input[2]-2]) + np.prod([conf.batchsize, conf.dim_output])

print("Total prams: ", num_param)
print("Input size(byte): ", num_input * 4)
print("Forward/backward(byte): ", 2 * num_hidden_output * 4) # https://github.com/sksq96/pytorch-summary/issues/51
print("Params size(byte): ", num_param * 4)

Layer (type:depth-idx)                   Output Shape              Param #                   Kernel Shape
CNN                                      [128, 10]                 --                        --
├─Conv2d: 1-1                            [128, 8, 222, 222]        224                       [3, 3]
├─AvgPool2d: 1-2                         [128, 8, 111, 111]        --                        2
├─Linear: 1-3                            [128, 10]                 985,690                   --
Total params: 985,914
Trainable params: 985,914
Non-trainable params: 0
Total mult-adds (G): 1.54
Input size (MB): 77.07
Forward/backward pass size (MB): 403.74
Params size (MB): 3.94
Estimated Total Size (MB): 484.76
Total prams:  985914
Input size(byte):  77070336
Forward/backward(byte):  403744768
Params size(byte):  3943656


torchinfoを利用して、定義したmodelからパラメータ数や中間層出力の数を自動的に計算して、理論値を計算できる。

In [7]:
def is_memoryless(class_name: str) -> bool:
    ''' Return True if the class is memoryless type.
    Activations, normalizations and dropouts perform in-place updates by default
    and does not require additional memory.
    '''
    return any((class_name == "ReLU",
           class_name == "LeakyReLU",
           class_name == "Sigmoid",
           class_name == "Tanh",
           class_name == "ELU",
           class_name == "GLU",
           class_name == "PReLU",
           class_name == "GELU",
           class_name == "Mish",
           class_name == "Softmin",
           class_name == "Softmax",
           class_name == "Softmax2d",
           class_name == "BatchNorm1d",
           class_name == "BatchNorm2d",
           class_name == "BatchNorm3d",
           class_name == "Dropout",
           class_name == "Dropout1d",
           class_name == "Dropout2d",
           class_name == "Dropout3d",
           class_name == "AlphaDropout"))

def print_memory_estimate2(
    model: nn.Module, 
    dim_input: list, 
    moment: int, 
    ddp: int=1, 
    mixed_pre: float = 1):
    '''Print theoretical memory usage.
    
    Parameters
    ----------
    model: 
    dim_input: Shape of input data including batch size. e.g. [batch size, channel, width, height]
    moment: Moment use for optimization. SGD: 0, Adagrad, RMSprop: 1, Adam: 2
    ddp: Multiple GPU use. Distributed data parallel: 2, Not: 1
    mixed_pre: Forward outputs memory saving by Mixed precision: 0.5, Not: 1
    '''
    info = summary(model, dim_input, verbose=0)
    dim_output = info.summary_list[-1].output_size[1:]

    num_param = 0
    num_output_shape = 0
    last_layer = len(info.summary_list) -1
    # print("#, Class, Leaf, Memoryless, Output")
    for i, layer in enumerate(info.summary_list):
        # print(f"{i}, {layer.class_name}, {layer.is_leaf_layer}, {is_memoryless(layer.class_name)}, {layer.output_size}")
        if layer.is_leaf_layer:
            num_param += layer.trainable_params
            if i != last_layer and not is_memoryless(layer.class_name):
                num_output_shape += np.prod(layer.output_size)
    
    mem_data = (np.prod(dim_input) + np.prod(dim_output)) * 4
    mem_weight = num_param * 4
    mem_weight_grad = mem_weight * (ddp + moment)
    mem_forward_output = num_output_shape * 4 * mixed_pre
    mem_output_gradient = mem_forward_output + mem_data
    mem_training = mem_data + mem_weight + mem_forward_output + mem_weight_grad + mem_output_gradient
    mem_inference = mem_data + mem_weight + mem_forward_output

    print(f"Data(MiB): {mem_data/1024**2:.1f}")
    print(f"Weight(MiB): {mem_weight/1024**2:.1f}")
    print(f"Forward output(MiB): {mem_forward_output/1024**2:.1f}")
    print(f"Weight gradient(MiB): {mem_weight_grad/1024**2:.1f}")
    print(f"Output gradient(MiB): {mem_output_gradient/1024**2:.1f}")
    print(f"Total for training(MiB): {mem_training/1024**2:.1f}")
    print(f"Total for inference(MiB): {mem_inference/1024**2:.1f}")


print_memory_estimate2(model_cnn, [conf.batchsize] + conf.dim_input, 
                       conf.moment, conf.ddp, conf.mixed_pre)

Data(MiB): 73.5
Weight(MiB): 3.8
Forward output(MiB): 240.6
Weight gradient(MiB): 3.8
Output gradient(MiB): 314.1
Total for training(MiB): 635.8
Total for inference(MiB): 317.9


## VGG16

In [8]:
import torchvision.models as models

model_vgg16 = models.vgg16_bn(weights=None)
model_vgg16.classifier[6] = nn.Linear(model_vgg16.classifier[6].in_features, 10) # Change the num of final output features to 10
# print(model_vgg16)

# result = summary(model_vgg16, [conf.batchsize] + conf.dim_input,
#                 col_names=("output_size", "num_params", "kernel_size"))
# print(result)

print("=== Estimated ===")
print_memory_estimate2(model_vgg16, [conf.batchsize] + conf.dim_input, 
                       conf.moment, conf.ddp, conf.mixed_pre)

print("=== Real ===")
train(model_vgg16, [conf.datasize] + conf.dim_input, conf.batchsize, conf.num_epochs,
      conf.optim, conf.device)

=== Estimated ===
Data(MiB): 73.5
Weight(MiB): 512.4
Forward output(MiB): 7378.5
Weight gradient(MiB): 512.4
Output gradient(MiB): 7452.0
Total for training(MiB): 15928.7
Total for inference(MiB): 7964.4
=== Real ===
Initial: allocated = 537.1 MiB, reserved = 678.0MiB, max allocated = 537.1 MiB, used = 5077.4 MiB
Model: allocated = 537.1 MiB, reserved = 678.0MiB, max allocated = 537.1 MiB, used = 5077.4 MiB


[Epoch 1]: 100%|██████████| 10/10 [00:03<00:00,  2.61it/s]


Train: allocated = 1125.1 MiB, reserved = 18752.0MiB, max allocated = 17231.9 MiB, used = 23131.2 MiB


[Epoch 2]: 100%|██████████| 10/10 [00:04<00:00,  2.46it/s]


Train: allocated = 1125.1 MiB, reserved = 18752.0MiB, max allocated = 17231.9 MiB, used = 23131.1 MiB


[Epoch 3]: 100%|██████████| 10/10 [00:04<00:00,  2.45it/s]

Train: allocated = 1125.1 MiB, reserved = 18752.0MiB, max allocated = 17231.9 MiB, used = 23131.1 MiB
Final: allocated = 1125.1 MiB, reserved = 18752.0MiB, max allocated = 17231.9 MiB, used = 23131.1 MiB


VGG16では、理論値が15929MiB、実測値が17232MiB、誤差は7.6%であった。

## ResNet50

In [9]:
model_rn50 = models.resnet50(weights=None)
model_rn50.fc = nn.Linear(model_rn50.fc.in_features, 10) # Change the num of final output features to 10
# print(model_rn50)

# result = summary(model_rn50, [conf.batchsize] + conf.dim_input,
#                 col_names=("output_size", "num_params", "kernel_size"))
# print(result)

print("=== Estimated ===")
print_memory_estimate2(model_rn50, [conf.batchsize] + conf.dim_input, 
                       conf.moment, conf.ddp, conf.mixed_pre)

print("=== Real ===")
train(model_rn50, [conf.datasize] + conf.dim_input, conf.batchsize, conf.num_epochs,
      conf.optim, conf.device)

=== Estimated ===
Data(MiB): 73.5
Weight(MiB): 89.8
Forward output(MiB): 5525.8
Weight gradient(MiB): 89.8
Output gradient(MiB): 5599.3
Total for training(MiB): 11378.0
Total for inference(MiB): 5689.0
=== Real ===
Initial: allocated = 1141.6 MiB, reserved = 7160.0MiB, max allocated = 1141.6 MiB, used = 11566.7 MiB
Model: allocated = 1141.6 MiB, reserved = 7160.0MiB, max allocated = 1141.6 MiB, used = 11566.7 MiB


[Epoch 1]: 100%|██████████| 10/10 [00:01<00:00,  5.45it/s]


Train: allocated = 1307.0 MiB, reserved = 12982.0MiB, max allocated = 11738.2 MiB, used = 17392.2 MiB


[Epoch 2]: 100%|██████████| 10/10 [00:02<00:00,  4.93it/s]


Train: allocated = 1307.0 MiB, reserved = 12982.0MiB, max allocated = 11738.2 MiB, used = 17392.4 MiB


[Epoch 3]: 100%|██████████| 10/10 [00:02<00:00,  4.92it/s]

Train: allocated = 1307.0 MiB, reserved = 12982.0MiB, max allocated = 11738.2 MiB, used = 17380.6 MiB
Final: allocated = 1307.0 MiB, reserved = 12982.0MiB, max allocated = 11738.2 MiB, used = 17380.6 MiB


ResNet50では、理論値が11378MiB、実測値が11738MiB、誤差は3.1%であった。

In [15]:
(17217 - 13290) / 17217

0.22808851716326886

## Visiont Transformer

In [13]:
model_vit = models.vit_b_16(weights=None)
model_vit.heads[0] = nn.Linear(model_vit.heads[0].in_features, 10) # Change the num of final output features to 10
print(model_vit)

result = summary(model_vit, [conf.batchsize] + conf.dim_input,
                col_names=("output_size", "num_params", "kernel_size"))
print(result)

print("=== Estimated ===")
print_memory_estimate2(model_vit, [conf.batchsize] + conf.dim_input, 
                       conf.moment, conf.ddp, conf.mixed_pre)

print("=== Real ===")
train(model_vit, [conf.datasize] + conf.dim_input, conf.batchsize, conf.num_epochs,
      conf.optim, conf.device)

VisionTransformer(
  (conv_proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
  (encoder): Encoder(
    (dropout): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (encoder_layer_0): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_attention): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (dropout): Dropout(p=0.0, inplace=False)
        (ln_2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=3072, out_features=768, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (encoder_layer_1): EncoderBlock(
        (ln_1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (self_a

[Epoch 1]: 100%|██████████| 10/10 [00:04<00:00,  2.14it/s]


Train: allocated = 1964.8 MiB, reserved = 18238.0MiB, max allocated = 17217.2 MiB, used = 22615.8 MiB


[Epoch 2]: 100%|██████████| 10/10 [00:05<00:00,  1.93it/s]


Train: allocated = 1964.8 MiB, reserved = 18238.0MiB, max allocated = 17217.2 MiB, used = 22608.0 MiB


[Epoch 3]: 100%|██████████| 10/10 [00:05<00:00,  1.93it/s]

Train: allocated = 1964.8 MiB, reserved = 18238.0MiB, max allocated = 17217.2 MiB, used = 22619.7 MiB
Final: allocated = 1964.8 MiB, reserved = 18238.0MiB, max allocated = 17217.2 MiB, used = 22619.7 MiB


In [14]:
print_memory_torch("")

: allocated = 1891.3 MiB, reserved = 18238.0MiB, max allocated = 17217.2 MiB, used = 22622.5 MiB
